In [1]:
import hopsy
import PolyRound
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
original_polytope = PolyRound.static_classes.parse_sbml_stoichiometry.StoichiometryParser.parse_sbml_cobrapy("../extern/hops/resources/e_coli_core/e_coli_core.xml")
polytope = PolyRound.api.PolyRoundApi.simplify_polytope(original_polytope)
polytope = PolyRound.api.PolyRoundApi.transform_polytope(polytope)
polytope.normalize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-22


In [3]:
replicates = 3
n_chains = 20

max_log_n_samples = 8
thin = 1
times = {}

In [4]:
p = hopsy.round(hopsy.Problem(polytope.A, polytope.b))

for i in range(replicates):

    s = hopsy.compute_chebyshev_center(p)
    markov_chains = [
        hopsy.MarkovChain(problem=p, starting_point=s)
        for i in range(n_chains)
    ]
    rngs = [hopsy.RandomNumberGenerator(i) for i in range(n_chains)]

In [ ]:
for n in range(1, max_log_n_samples):
    times[n] = []
    for i in range(replicates):
        start = time.perf_counter()
        _, samples = hopsy.sample(
            markov_chains, rngs, n_samples=int(10**n/10), thinning=10, n_procs=n_chains
        )
        del samples
        end = time.perf_counter()
        times[n].append(end-start)
    times[n] = np.mean(times[n])
    print(n, ":", np.mean(times[n]))

1 : 0.2198675799979052
2 : 0.2740902796698113
3 : 0.20827976200477374
4 : 0.22025225133499285
5 : 0.5074554303331146
6 : 3.1844185713319653


In [ ]:
print(times)

In [ ]:
plt.plot(times.keys(), times.values())

In [ ]:
times_ = {}
for n in range(1, max_log_n_samples-1):
    times_[n] = []
    for i in range(replicates):
        start = time.perf_counter()
        _, samples = hopsy.sample(
            markov_chains, rngs, n_samples=int(10**n), thinning=1, n_procs=n_chains
        )
        del samples
        end = time.perf_counter()
        times_[n].append(end-start)
    times_[n] = np.mean(times_[n])
    print(n, ":", np.mean(times_[n]))

In [ ]:

plt.plot(times_.keys(), times_.values())